In [1]:
import os
from typing import Tuple

import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import LearningRateScheduler # Adaptive Lernrate: Passt jede Epoche die Lernrate an
from tensorflow.keras.callbacks import ReduceLROnPlateau # Passt Lernrate an (factor), wenn Training stagniert (Min_delta)
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import ELU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import MaxPool2D, GlobalAveragePooling2D
from tensorflow.keras.layers import ReLU
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam # Hat auch eine adaptive Lernrate

from tf_utils.dogscatsDataAdvanced import DOGSCATS
from tf_utils.callbacks import LRTensorBoard
from tf_utils.callbacks import schedule_fn
from tf_utils.callbacks import schedule_fn2
from tf_utils.callbacks import schedule_fn3
from tf_utils.callbacks import schedule_fn4

In [2]:
np.random.seed(0)
tf.random.set_seed(0)

In [3]:
LOGS_DIR = os.path.abspath('C:/Selbststudium/Udemy/Udemy_Tensorflow/logs/DOGSCATS/')
if not os.path.exists(LOGS_DIR):
    os.mkdir(LOGS_DIR)

In [4]:
def build_model(
    img_shape: Tuple[int, int, int],
    num_classes: int,
    optimizer: tf.keras.optimizers.Optimizer,
    learning_rate: float,
    filter_block_1: int,
    kernel_size_block_1: int,
    filter_block_2: int,
    kernel_size_block_2: int,
    filter_block_3: int,
    kernel_size_block_3: int,
    dense_layer_size: int,
    kernel_initializer: tf.keras.initializers.Initializer,
    activation_cls: tf.keras.layers.Activation,
    dropout_rate: float,
    use_batch_normalization: bool,
    use_dense: bool,
    use_global_pooling: bool
) -> Model:
    input_img = Input(shape=img_shape)

    x = Conv2D(
        filters=filter_block_1, 
        kernel_size=kernel_size_block_1, 
        padding='same', 
        kernel_initializer=kernel_initializer
    )(input_img)
    if use_batch_normalization:
        x = BatchNormalization()(x)
    x = activation_cls(x)
    x = Conv2D(
        filters=filter_block_1, 
        kernel_size=kernel_size_block_1, 
        padding='same', 
        kernel_initializer=kernel_initializer
    )(x)
    if use_batch_normalization:
        x = BatchNormalization()(x)
    if dropout_rate:
        x = Dropout(rate=dropout_rate)(x)
    x = activation_cls(x)
    x = MaxPool2D()(x)

    x = Conv2D(
        filters=filter_block_2, 
        kernel_size=kernel_size_block_2, 
        padding='same', 
        kernel_initializer=kernel_initializer
    )(x)
    if use_batch_normalization:
        x = BatchNormalization()(x)
    x = activation_cls(x)
    x = Conv2D(
        filters=filter_block_2, 
        kernel_size=kernel_size_block_2, 
        padding='same', 
        kernel_initializer=kernel_initializer
    )(x)
    if use_batch_normalization:
        x = BatchNormalization()(x)
    if dropout_rate:
        x = Dropout(rate=dropout_rate)(x)
    x = activation_cls(x)
    x = MaxPool2D()(x)

    x = Conv2D(
        filters=filter_block_3, 
        kernel_size=kernel_size_block_3, 
        padding='same', 
        kernel_initializer=kernel_initializer
    )(x)
    if use_batch_normalization:
        x = BatchNormalization()(x)
    x = activation_cls(x)
    x = Conv2D(
        filters=filter_block_3, 
        kernel_size=kernel_size_block_3, 
        padding='same', 
        kernel_initializer=kernel_initializer
    )(x)
    if use_batch_normalization:
        x = BatchNormalization()(x)
    if dropout_rate:
        x = Dropout(rate=dropout_rate)(x)
    x = activation_cls(x)
    x = MaxPool2D()(x)

    if use_global_pooling:
        x = GlobalAveragePooling2D()(x)
    else:
        x = Flatten()(x)
    if use_dense:
        x = Dense(
            units=dense_layer_size, 
            kernel_initializer=kernel_initializer
        )(x)
        if use_batch_normalization:
            x = BatchNormalization()(x)
        x = activation_cls(x)
    x = Dense(
        units=num_classes, 
        kernel_initializer=kernel_initializer
    )(x)
    y_pred = Activation('softmax')(x)

    # Jetzt muss noch ein Modell Objekt mit eben obiger Struktur erstellt werden
    model = Model(
        inputs = [input_img],
        outputs = [y_pred]
    )
    
    opt = optimizer(learning_rate=learning_rate)
    
    model.compile(
        loss='categorical_crossentropy', # wird bei Kategorie-Problemen mit mehr als 2 Klassen genommen
        optimizer=opt,
        metrics=['accuracy']
    )

    return model

In [5]:
data = DOGSCATS()

train_dataset = data.get_train_set()
val_dataset = data.get_val_set()
test_dataset = data.get_test_set()

img_shape = data.img_shape
num_classes = data.num_classes

epochs = 40
batch_size = 128

# Best model params
params = {
    "optimizer": Adam,
    "learning_rate": 0.001,
    "filter_block_1": 32,
    "kernel_size_block_1": 3,
    "filter_block_2": 64,
    "kernel_size_block_2": 3,
    "filter_block_3": 128,
    "kernel_size_block_3": 3,
    "dense_layer_size": 128,
    "kernel_initializer": "GlorotUniform",
    "activation_cls": ReLU(),
    "dropout_rate": 0.0,
    "use_batch_normalization": True,
    "use_dense": False,
    "use_global_pooling": False
}

In [6]:
# In diesem Video bzw. genau diesem Abschnitt wird Batch Normalization getestet

model = build_model(
    img_shape,
    num_classes,
    **params
)

use_pool = params["use_global_pooling"] # Damit der f-String leserlicher wird
use_dense = params["use_dense"]
model_log_dir = os.path.join(LOGS_DIR, f"model_Global_{use_pool}_Dense_{use_dense}")

tb_callback = TensorBoard(
    log_dir=model_log_dir,
    histogram_freq=0,
    profile_batch=0,
    write_graph=0
)

lrs_callback = LearningRateScheduler(
    schedule=schedule_fn2,
    verbose=1
)

plateau_callback = ReduceLROnPlateau(
    monitor="val_accuracy",
    factor=0.99,
    patience=3,
    verbose=1,
    min_lr=1e-5
)

lr_callback = LRTensorBoard(
    log_dir=model_log_dir,
    histogram_freq=0,
    profile_batch=0,
    write_graph=0
)

es_callback = EarlyStopping(
    monitor="val_accuracy",
    patience=30,
    verbose=1,
    restore_best_weights=True,
    min_delta=0.0005
)

model.fit(
    train_dataset,
    verbose=1,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=[es_callback, lrs_callback, lr_callback],
    validation_data=val_dataset,
)


Epoch 00001: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/40
105/105 [==============================] - 15s 94ms/step - loss: 0.8211 - accuracy: 0.7010 - val_loss: 0.7886 - val_accuracy: 0.4973 - lr: 0.0010 - learning_rate: 0.0010

Epoch 00002: LearningRateScheduler setting learning rate to 0.001.
Epoch 2/40
105/105 [==============================] - 9s 86ms/step - loss: 0.4530 - accuracy: 0.7899 - val_loss: 1.0018 - val_accuracy: 0.4974 - lr: 0.0010 - learning_rate: 0.0010

Epoch 00003: LearningRateScheduler setting learning rate to 0.001.
Epoch 3/40
105/105 [==============================] - 9s 86ms/step - loss: 0.3616 - accuracy: 0.8418 - val_loss: 0.7288 - val_accuracy: 0.5645 - lr: 0.0010 - learning_rate: 0.0010

Epoch 00004: LearningRateScheduler setting learning rate to 0.001.
Epoch 4/40
105/105 [==============================] - 9s 87ms/step - loss: 0.2955 - accuracy: 0.8755 - val_loss: 0.7575 - val_accuracy: 0.6687 - lr: 0.0010 - learning_rate: 0.0010

Epoch 